In [3]:
import numpy as np
import os
from ase.io import read


In [ ]:
def build_hamiltonian(
    onsite_params: list[float],
    nearest_neighbor_t: float,
    second_nearest_neighbor_t: float,
) -> np.ndarray:
    """
    Create a Hermitian Hamiltonian matrix H for a system with n sites.

    Parameters
    ----------
    onsite_params : list[float]
        List of diagonal elements (onsite energies) for each site.
    nearest_neighbor_t : float
        Hopping parameter for nearest neighbors (t).
    second_nearest_neighbor_t : float
        Hopping parameter for second nearest neighbors (t').

    Returns
    -------
    np.ndarray
        n x n Hermitian Hamiltonian matrix H where n is the length of onsite_params.

    Raises
    ------
    ValueError
        If onsite_params is empty or if hopping parameters are not finite.
    """
    n = len(onsite_params)

    if n == 0:
        raise ValueError("onsite_params must contain at least one element.")

    if not np.isfinite(nearest_neighbor_t):
        raise ValueError("nearest_neighbor_t must be a finite number.")

    if not np.isfinite(second_nearest_neighbor_t):
        raise ValueError("second_nearest_neighbor_t must be a finite number.")

    # Initialize an n x n zero matrix
    H = np.zeros((n, n), dtype=complex)

    # Set diagonal elements
    for i in range(n):
        H[i, i] = onsite_params[i]

    # Set off-diagonal elements for t and t'
    for i in range(n - 1):
        H[i, i + 1] = nearest_neighbor_t
        H[i + 1, i] = np.conj(nearest_neighbor_t)  # Ensure Hermitian property

    for i in range(n - 2):
        H[i, i + 2] = second_nearest_neighbor_t
        H[i + 2, i] = np.conj(second_nearest_neighbor_t)  # Ensure Hermitian property

    return H

def compute_green_element(E: float,
                          H: np.ndarray,
                          GammaL: np.ndarray,
                          GammaR: np.ndarray,
                          SigmaD: np.ndarray,
                          eta: float,
                          l: int,
                          r: int) -> complex:
    """
    Computes G_D(z) and returns the (l,r) element.

    G_D(z) = [ z - H + i Γ_L/2 + i Γ_R/2 - Σ_D(z) ]^{-1}
    with z = E + i eta.
    """
    n = H.shape[0]
    I = np.eye(n, dtype=complex)
    z = E + 1j * eta

    A = (z * I) - H.astype(complex) + 0.5j * GammaL.astype(complex) + 0.5j * GammaR.astype(complex) - SigmaD.astype(complex)
    GD = np.linalg.inv(A)
    return GD[l, r]


def compute_transmission(energies: np.ndarray,
                              H: np.ndarray,
                              gamma_L: np.ndarray,
                              gamma_R: np.ndarray,
                              sigma_D: np.ndarray,
                              eta: float = 1e-2,
                              left_index: int = 0,
                              right_index: int = -1) -> np.ndarray:
    """
    Computes T(E) using Eq. (33) in "Strongly correlated physics in open-shell organic molecules":
      T(E) = Γ^2 |G_lr(E)|^2

    Interpretation used:
      Γ^2 = Γ_L(l,l) * Γ_R(r,r)
    where l is left_index and r is right_index.
    """
    if H.ndim != 2 or H.shape[0] != H.shape[1]:
        raise ValueError(f"H must be square (n,n); got {H.shape}")
    n = H.shape[0]

    nE = energies.size
    if sigma_D.shape != (nE, n, n):
        raise ValueError(f"sigma_D must be (nE,n,n)={(nE,n,n)}; got {sigma_D.shape}")

    l = left_index if left_index >= 0 else n + left_index
    r = right_index if right_index >= 0 else n + right_index
    if not (0 <= l < n and 0 <= r < n):
        raise ValueError(f"Invalid (l,r)=({left_index},{right_index}) -> ({l},{r}) for n={n}")

    T = np.empty(nE, dtype=float)

    for eidx, E in enumerate(energies):

        # G_lr from Eq. (31)
        Glr = compute_green_element(E, H, gamma_L, gamma_R, sigma_D[eidx], eta, l, r)

        # Eq. (33): Γ^2 |G_lr|^2 with Γ^2 = Γ_L(l,l) Γ_R(r,r)
        gamma_sq = (gamma_L[l, l] * gamma_R[r, r]).real
        T[eidx] = max(0.0, float(gamma_sq * (abs(Glr) ** 2)))  # clamp tiny negative from rounding

    return T


def build_ppp_matrix(
    system_root: str,
    num_sites: int,
    *,
    U_onsite: float = 2.0,
    coulomb_evA: float = 14.397,
    symbols: tuple[str, ...] = ("C", "N"),
    xyz_path: str = "./scatt.xyz",
    symmetrize: bool = True,
) -> np.ndarray:
    """
    Build and return a PPP / Ohno-type interaction matrix U (num_sites x num_sites).

    Parameters
    ----------
    system_root : str
        Root directory containing the geometry.
    num_sites : int
        Number of sites (matrix dimension).
    U_onsite : float
        Onsite interaction U (eV).
    coulomb_evA : float
        Coulomb conversion factor e^2/(4πϵ0) in eV·Å.
    symbols : tuple[str, ...]
        Chemical symbols used as sites (default: C, N).
    xyz_path : str
        Relative path to the XYZ geometry file.
    symmetrize : bool
        If True, enforce U = (U + U.T)/2.

    Returns
    -------
    np.ndarray
        PPP interaction matrix U (eV).

    Raises
    ------
    FileNotFoundError
        If the XYZ file is missing.
    ValueError
        If too few atoms are available or parameters are unphysical.
    """
    if num_sites <= 0:
        raise ValueError(f"num_sites must be positive; got {num_sites}")

    if U_onsite <= 0:
        raise ValueError(f"U_onsite must be positive; got {U_onsite}")

    # ---------------------------
    # PPP parameter
    # ---------------------------
    alpha_ppp = (U_onsite / coulomb_evA) ** 2  # 1/Å^2

    # ---------------------------
    # Load geometry
    # ---------------------------
    xyz_full_path = os.path.join(system_root, xyz_path)
    if not os.path.isfile(xyz_full_path):
        raise FileNotFoundError(f"Missing geometry file: {xyz_full_path}")

    atoms = read(xyz_full_path)
    keep = np.isin(atoms.get_chemical_symbols(), list(symbols))
    site_atoms = atoms[keep]

    if len(site_atoms) < num_sites:
        raise ValueError(
            f"Found only {len(site_atoms)} {symbols} atoms, "
            f"but num_sites={num_sites} was requested."
        )

    positions = site_atoms.get_positions()[:num_sites]  # Å

    # ---------------------------
    # Distance matrix r_ij
    # ---------------------------
    disp = positions[:, None, :] - positions[None, :, :]
    r = np.linalg.norm(disp, axis=-1)

    # ---------------------------
    # PPP interaction matrix
    # ---------------------------
    U = U_onsite / np.sqrt(1.0 + alpha_ppp * r**2)
    np.fill_diagonal(U, U_onsite)

    if symmetrize:
        U = 0.5 * (U + U.T)

    return U

In [13]:
nsites = 5
nearest_neighbor_t = 0.5
second_nearest_neighbor_t = 0.0
onsite_params = [0.5, 0.25, 0.25, 0.25, 0.5]
H = build_hamiltonian(onsite_params, nearest_neighbor_t, second_nearest_neighbor_t)
np.save("output/hamiltonian.npy", H)

In [10]:
eigenvalues, eigenvectors = np.linalg.eigh(H)
eigenvalues

array([-0.58200782, -0.1403882 ,  0.41358338,  0.8903882 ,  1.16842445])

In [ ]:
gamma_L = np.zeros((nsites, nsites), dtype=complex)
gamma_R = np.zeros((nsites, nsites), dtype=complex)
gamma_L[0,0] = 0.05
gamma_R[-1,-1] = 0.05
np.save("output/gamma_L.npy", gamma_L)
np.save("output/gamma_R.npy", gamma_R)

array([[0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j],
       [0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j],
       [0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j],
       [0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j],
       [0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j, 0.05+0.j]])

In [ ]:
occupancies = np.ones(nsites)
np.save("output/occupancies.npy", occupancies)

In [ ]:
U_ppp = build_ppp_matrix(
    system_root="output",
    num_sites=nsites,
    U_onsite=4.62,
    coulomb_evA=14.397,
    symbols=("C", "N"),
    xyz_path="scatt.xyz",
    symmetrize=True,
)
np.savetxt("output/U_ppp.txt", U_ppp)

In [ ]:
energies = np.load("output/energies.npy")
H = np.load("output/hamiltonian.npy")
gamma_L = np.load("output/gamma_L.npy")
gamma_R = np.load("output/gamma_R.npy")
sigma_D = np.load("output/ed/self_energy.npy")

# Notebook-local compute_transmission returns total only.
T_total = compute_transmission(energies, H, gamma_L, gamma_R, sigma_D)
T_elastic = T_total
T_inelastic = np.zeros_like(T_total)

np.save("output/ed/ET.npy", np.array([energies, T_elastic, T_inelastic, T_total]))